In [5]:
import pandas as pd
#pd.set_option('display.max_columns', 500)
#pd.set_option('display.max_rows', 500)
import pickle

import sys
import numpy as np
np.set_printoptions(threshold=sys.maxsize)

import matplotlib.pyplot as plt
from PIL import Image, ImageEnhance, ImageDraw,ImageFont

import arviz as az
import pymc3 as pm
print(pm.__version__)
import theano.tensor as tt


import os
import re
import glob
import random

from sklearn import preprocessing

from skimage import measure, restoration,morphology
from skimage import io, filters, measure, color, img_as_ubyte
from skimage.draw import disk
from skimage import measure, restoration,morphology

RANDOM_SEED = 8927
np.random.seed(RANDOM_SEED)


%config InlineBackend.figure_format = 'retina'

3.11.5


Model Varying slops - start signal - finish signal (chapter 14 caffe example) and distance matrix

In [6]:
path = '/data/kanferg/Images/Pex_project/Basyein_classifcation/output_lab_meeting/training_data'
table_train_mix_varyin_effect = pd.read_csv(os.path.join(path,'table_train_mix_varyin_effect.csv'))
table_test_mix_varyin_effect = pd.read_csv(os.path.join(path,'table_test_mix_varyin_effect.csv'))

In [7]:
table_train_mix_varyin_effect

,Unnamed: 0.1,index,label,area,centroid-0,centroid-1,pheno_class,image_number,Unnamed: 0,image_signal,raius_list,image_group,fov_number,image_t_o,image_t_
0,0,16800,210,2557,346.347282,1769.860774,1,0,16800,1729.525616,0,210,0,1729.525616,486.579585
1,0,17680,221,2535,359.369625,138.584615,1,0,17680,3279.715582,0,221,0,3279.715582,289.120316
2,0,18960,237,2836,384.560296,1745.599788,1,0,18960,1854.755994,0,237,0,1854.755994,491.741185
3,0,57280,716,3521,1241.553536,354.123544,1,0,57280,2022.447600,0,716,0,2022.447600,319.905993
4,0,74160,927,4883,1648.965800,713.038501,1,0,74160,1476.749744,0,927,0,1476.749744,148.775548
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,0,39040,488,1758,924.425484,2488.064278,0,2,39040,473.081911,0,488,2,473.081911,134.526166
137,0,79920,999,3746,1907.332355,2238.163107,0,2,79920,3791.721303,0,999,2,3791.721303,154.540577
138,0,11680,146,4218,255.200332,1428.484353,0,0,11680,6142.033428,0,146,0,6142.033428,332.568516
139,0,6000,75,6570,116.541553,1300.870472,0,0,6000,862.441857,0,75,0,862.441857,125.018417


In [23]:
table_mix_varyin_effect = table_train_mix_varyin_effect
df_varying_effect = table_mix_varyin_effect.loc[:,['label','image_number','pheno_class','centroid-0','image_t_o','image_t_']]
df_varying_effect = pd.melt(frame=df_varying_effect,id_vars=['label','image_number','pheno_class','centroid-0',],var_name='t_var').sort_values('label').reset_index()
df_varying_effect['t_var'] = df_varying_effect.apply(lambda x: 1 if x['t_var'] == 'image_t_o' else 0,axis = 1) 
# reset table
uniqe__label = np.unique(df_varying_effect.label.values)
df_varying_effect['reset_lable'] = None
for i in range(len(uniqe__label)):
    df_varying_effect.loc[df_varying_effect['label'] == uniqe__label[i],'reset_lable'] = int(i)
# Change the set  
df_varying_effect["reset_lable"] = pd.to_numeric(df_varying_effect["reset_lable"])
df_varying_effect["reset_lable"].values
# change from 16 bit to duble
df_varying_effect['value'] = df_varying_effect['value'].values/2**16

In [27]:
df_varying_effect

,index,label,image_number,pheno_class,centroid-0,t_var,value,reset_lable
0,114,30,4,0,31.764552,1,0.055308,0
1,255,30,4,0,31.764552,0,0.002278,0
2,200,31,6,1,57.568163,0,0.002609,1
3,59,31,6,1,57.568163,1,0.036111,1
4,201,33,6,1,70.716582,0,0.002609,2
...,...,...,...,...,...,...,...,...
277,118,1646,1,0,2538.567262,1,0.036384,133
278,130,1694,3,0,2733.467162,1,0.029694,134
279,271,1694,3,0,2733.467162,0,0.003296,134
280,103,1750,3,0,2807.222587,1,0.032861,135


In [65]:
id_arr = df_varying_effect["reset_lable"].values
Nid = len(np.unique(df_varying_effect["reset_lable"].values))
class_arr = df_varying_effect["pheno_class"].values
NClass = 2
arr_y = df_varying_effect["value"].values
t_var = df_varying_effect['t_var'].values
print('id_arr{}'.format(len(id_arr)))
print('Nid{}'.format(Nid))
print('arr_y{}'.format(len(arr_y)))
print('t_var{}'.format(len(t_var)))

id_arr282
Nid136
arr_y282
t_var282


In [66]:
def model_factory_Varying_slops(Nid,id_arr):
    with pm.Model() as model:
        a = pm.Normal('a',0,0.5)
        b = pm.Normal('b',0,0.5)
        chol, Rho_, sigma_cafe = pm.LKJCholeskyCov(
            "chol_cov", n=2, eta=2, sd_dist=pm.Exponential.dist(0.1), compute_corr=True)
        ab_signal = pm.MvNormal(
            "ab_signal", mu=tt.stack([a, b]), chol=chol, shape=(Nid, 2))  # population of varying effects
        mu = ab_signal[id_arr, 0] + ab_signal[id_arr, 1] * df_varying_effect['t_var'].values
        sigma_within = pm.Exponential("sigma_within", 0.1)
        decay = pm.Normal("decay", mu=mu, sigma=sigma_within, observed=df_varying_effect["value"].values)
        return model

In [68]:
with model_factory_Varying_slops(Nid=2, id_arr=class_arr) as train_model:
    train_trace_Varying_slops = pm.sample(4000, tune=4000, target_accept=0.9,random_seed=RANDOM_SEED)
    idata_14_1 = az.from_pymc3(train_trace_Varying_slops)
az.summary(idata_14_1,round_to=2)   

/opt/conda/envs/3.11.5/lib/python3.9/site-packages/deprecat/classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [sigma_within, ab_signal, chol_cov, b, a]


Sampling 4 chains for 4_000 tune and 4_000 draw iterations (16_000 + 16_000 draws total) took 193 seconds.
There were 638 divergences after tuning. Increase `target_accept` or reparameterize.
There were 1207 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.7323475607647987, but should be close to 0.9. Try to increase the number of tuning steps.
There were 1061 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.7725514985684875, but should be close to 0.9. Try to increase the number of tuning steps.
There were 1044 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.745446940826597, but should be close to 0.9. Try to increase the number of tuning steps.
The number of effective samples is smaller than 10% for some parameters.
/opt/conda/envs/3.11.5

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
a,0.00,0.19,-0.40,0.40,0.00,0.00,2886.34,1728.07,1.01
b,0.04,0.23,-0.44,0.55,0.01,0.00,1945.55,1508.70,1.02
"ab_signal[0, 0]",0.00,0.00,0.00,0.01,0.00,0.00,477.15,982.84,1.02
"ab_signal[0, 1]",0.04,0.00,0.03,0.04,0.00,0.00,223.15,1224.52,1.02
"ab_signal[1, 0]",0.00,0.00,0.00,0.01,0.00,0.00,834.73,1499.15,1.02
"ab_signal[1, 1]",0.05,0.00,0.05,0.06,0.00,0.00,911.49,1200.59,1.01
chol_cov[0],0.41,1.11,0.00,1.48,0.02,0.01,215.27,306.05,1.02
chol_cov[1],0.03,0.82,-0.79,0.82,0.03,0.02,1443.08,1067.63,1.02
chol_cov[2],0.47,1.37,0.01,1.62,0.05,0.03,274.67,438.96,1.03
chol_cov_stds[0],0.41,1.11,0.00,1.48,0.02,0.01,215.27,306.05,1.02


In [ ]:
with model_factory(x=opening_opr__test,
                   y=id_paramaters__test,
                   z=id_test,
                   sig=signal__test) as test_model:
    # We first have to extract the learnt global effect from the train_trace
    df = pm.trace_to_dataframe(Nid=2, id_arr=class_arr,
                               varnames=["a","b","c",'sigma_within'],
                               include_transformed=True)
    p_post = pm.sample_posterior_predictive(trace=df.to_dict('records'),
                                            var_names=["a","b","c",'sigma_within'],samples=4000)

The cafe model works very bed
# model multilevel - non-centered (Chimmpanze)

In [41]:
path = '/data/kanferg/Images/Pex_project/Basyein_classifcation/output_lab_meeting/training_data'
table_mix = pd.read_csv(os.path.join(path,'table_sum_train.csv'))

# Change the set
table_mix['reset_lable'] = 0
uniqe__label = np.unique(table_mix['centroid-0'].values)
for i in range(len(uniqe__label)):
    table_mix.loc[table_mix['centroid-0'] == uniqe__label[i],'reset_lable'] = int(i)

table_mix["reset_lable"] = pd.to_numeric(table_mix["reset_lable"])
table_mix

,Unnamed: 0.1,index,label,area,centroid-0,centroid-1,pheno_class,image_number,Unnamed: 0,image_signal,raius_list,image_group,fov_number,reset_lable
0,0,16800,210,2557,346.347282,1769.860774,1,0,16800,1729.525616,0,210,0,23
1,1,16801,210,2557,346.347282,1769.860774,1,0,16801,1701.712945,1,210,0,23
2,2,16802,210,2557,346.347282,1769.860774,1,0,16802,1672.143919,2,210,0,23
3,3,16803,210,2557,346.347282,1769.860774,1,0,16803,1634.346109,3,210,0,23
4,4,16804,210,2557,346.347282,1769.860774,1,0,16804,1602.489245,4,210,0,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11275,75,41435,517,3004,802.925433,1096.210719,0,3,41435,233.302597,75,517,3,52
11276,76,41436,517,3004,802.925433,1096.210719,0,3,41436,232.611518,76,517,3,52
11277,77,41437,517,3004,802.925433,1096.210719,0,3,41437,231.931092,77,517,3,52
11278,78,41438,517,3004,802.925433,1096.210719,0,3,41438,231.405459,78,517,3,52


In [75]:
opr_arr_uniqe = np.linspace(1,80,80,dtype=int,endpoint=True)/100
Nopr = len(opr_arr_uniqe)
opr_arr = np.array(table_mix.raius_list.values)

# binary observation 
Nid_binary = 2
Zeros = len(table_mix.pheno_class.values) - np.sum(table_mix.pheno_class.values)
Ones = np.sum(table_mix.pheno_class.values)
id_arr_binary = np.concatenate((np.repeat(0,Zeros),np.repeat(1,Ones)))

# observation
idd_arr = table_mix["reset_lable"].values
Nidd = len(np.unique(idd_arr))

y = table_mix.image_signal.values/2**16
print('id_arr:{}'.format(len(id_arr)))
print('opening_opr:{}'.format(len(opr_arr)))
print('signal:{}'.format(len(y)))

id_arr:282
opening_opr:11280
signal:11280


In [76]:
y

array([0.02639047, 0.02596608, 0.02551489, 0.02493814, 0.02445205,
       0.02367996, 0.02301709, 0.02208763, 0.02158351, 0.02075552,
       0.02010021, 0.01976733, 0.01909466, 0.0188927 , 0.01864529,
       0.01784659, 0.01671651, 0.01548646, 0.0146199 , 0.01434   ,
       0.01362139, 0.01345212, 0.01328699, 0.0130827 , 0.01289115,
       0.01284616, 0.01279271, 0.01275631, 0.01271694, 0.01264263,
       0.012033  , 0.01198795, 0.01109151, 0.01043484, 0.01024297,
       0.01000132, 0.00989913, 0.0097584 , 0.00958392, 0.00952513,
       0.00951452, 0.00950329, 0.00949101, 0.00945912, 0.0094407 ,
       0.00943441, 0.00943537, 0.00942648, 0.00941488, 0.00940937,
       0.00939318, 0.00938779, 0.00938099, 0.00937803, 0.00936557,
       0.00929307, 0.0087506 , 0.00846219, 0.00804521, 0.00775525,
       0.00742462, 0.00739576, 0.00733719, 0.00681198, 0.0061431 ,
       0.00613979, 0.00613835, 0.00613753, 0.00613649, 0.00613535,
       0.00613451, 0.00613273, 0.00613107, 0.00613117, 0.00612

In [73]:
# def model_factory_multilevel_centered(Nopr,Nid,id_arr, arr_y):
#     with pm.Model() as model:
#         a = pm.Normal('a',0,0.05,shape = Nid)
#         sd_dist = pm.Exponential.dist(0.01)
#         chol_cb, _, _ = pm.LKJCholeskyCov("chol_cb", n=Nid, eta=4, sd_dist=sd_dist, compute_corr=True)
#         cb = pm.MvNormal("cb", mu=0.0, chol=chol_cb, shape=(Nopr,Nid))
#         mu = a[id_arr] + cb[opr_arr,id_arr]# linear model
#         sigma_within = pm.Exponential("sigma_within",0.1)  # prior stddev within image
#         signal = pm.Normal("signal", mu=mu, sigma=sigma_within, observed=arr_y)  # likelihood
#         return model

In [77]:
def model_factory_multilevel_centered(Nopr,Nid,id_arr, arr_y):
    with pm.Model() as model:
        sd_dist = pm.Exponential.dist(0.01)
        chol_cb, _, _ = pm.LKJCholeskyCov("chol_cb", n=Nid, eta=4, sd_dist=sd_dist, compute_corr=True)
        cb = pm.MvNormal("cb", mu=0.0, chol=chol_cb, shape=(Nopr,Nid))
        mu = cb[opr_arr,id_arr]# linear model
        sigma_within = pm.Exponential("sigma_within",0.1)  # prior stddev within image
        signal = pm.Normal("signal", mu=mu, sigma=sigma_within, observed=arr_y)  # likelihood
        return model

In [78]:
with model_factory_multilevel_centered(Nopr=Nopr,
                   Nid=Nidd,
                   id_arr=idd_arr,
                   arr_y=y) as train_model:
    train_trace_multilevel_centered = pm.sample(4000, tune=4000, target_accept=0.9,random_seed=RANDOM_SEED)

/opt/conda/envs/3.11.5/lib/python3.9/site-packages/deprecat/classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [sigma_within, cb, chol_cb]


/opt/conda/envs/3.11.5/lib/python3.9/site-packages/pymc3/step_methods/hmc/quadpotential.py:220: RuntimeWarning: divide by zero encountered in true_divide
  np.divide(1, self._stds, out=self._inv_stds)
/opt/conda/envs/3.11.5/lib/python3.9/site-packages/pymc3/step_methods/hmc/quadpotential.py:199: RuntimeWarning: invalid value encountered in multiply
  return np.multiply(self._var, x, out=out)
/opt/conda/envs/3.11.5/lib/python3.9/site-packages/pymc3/step_methods/hmc/quadpotential.py:220: RuntimeWarning: divide by zero encountered in true_divide
  np.divide(1, self._stds, out=self._inv_stds)
/opt/conda/envs/3.11.5/lib/python3.9/site-packages/pymc3/step_methods/hmc/quadpotential.py:199: RuntimeWarning: invalid value encountered in multiply
  return np.multiply(self._var, x, out=out)
/opt/conda/envs/3.11.5/lib/python3.9/site-packages/pymc3/step_methods/hmc/quadpotential.py:220: RuntimeWarning: divide by zero encountered in true_divide
  np.divide(1, self._stds, out=self._inv_stds)
/opt/cond

RuntimeError: Chain 2 failed.

In [81]:
(0.91*0.01)/((0.91*0.01)+(0.09*0.91))

0.1